# Import Necessary Packages

In [ ]:
#Packages Required
import pandas as pd
from bs4 import BeautifulSoup
import html
import emoji
import re
from datetime import datetime
from urllib.parse import unquote
import json
import io
import zstandard
import nltk
from nltk.stem import SnowballStemmer
from fuzzywuzzy import process, fuzz
import numpy as np
from scipy.stats import skew, kurtosis
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
import seaborn as sns
import statsmodels.api as sm

pd.options.mode.chained_assignment = None
pd.set_option('display.max_colwidth', None)
pd.set_option("display.precision", 3) #Set the default Pandas float precision to 2 decimals

In [ ]:
# Run only once
# Download NLTK resources (you only need to do this once)
nltk.download('punkt')

# Data Loader (Convert data from zst to dataframe)

## For Reddit Comments

In [ ]:
import json
import io
import zstandard
import pandas as pd
from datetime import datetime

# ============================================================
# Function Definitions
# ============================================================

def read_and_decode_zst(file_path):
    """
    Reads and decompresses a .zst file, then decodes the JSON data into a pandas DataFrame.
    """
    with open(file_path, 'rb') as file_handle:
        reader = zstandard.ZstdDecompressor().stream_reader(file_handle)
        data = reader.read().decode()
        data = pd.read_json(io.StringIO(data), lines=True)
    return data

def read_and_process_comment_zst(file_path, min_timestamp=1552288722):
    """
    Reads a .zst file line by line, extracts relevant data, and returns two DataFrames:
    one with valid data and one with errors.
    """
    selected_data = []
    error_data = []

    try:
        with open(file_path, 'rb') as file_handle:
            reader = zstandard.ZstdDecompressor().stream_reader(file_handle)
            text_reader = io.TextIOWrapper(reader, encoding='utf-8')
            for line in text_reader:
                try:
                    data_dict = json.loads(line)
                    created_utc = int(data_dict.get('created_utc', 0))
                    if created_utc >= min_timestamp:
                        selected_data.append({
                            'author': data_dict.get('author'),
                            'author_created_utc': data_dict.get('author_created_utc'),
                            'body': data_dict.get('body'),
                            'created_utc': data_dict.get('created_utc'),
                            'id': data_dict.get('id'),
                            'link_id': data_dict.get('link_id'),
                            'parent_id': data_dict.get('parent_id'),
                            'votes': data_dict.get('score'),
                            'retrieved_utc': data_dict.get('retrieved_on'),
                            'subreddit': data_dict.get('subreddit'),
                        })
                except Exception as e:
                    error_data.append({'error': str(e), 'data': line})
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None, None

    df_selected_data = pd.DataFrame(selected_data)
    df_error_data = pd.DataFrame(error_data)

    print(f'The earliest comment in the data is UTC {df_selected_data.created_utc.min()}') 
    print(f'The selected data has {len(df_selected_data)} rows.')
    print(f'The data with errors has {len(df_error_data)} rows.')

    return df_selected_data, df_error_data

def process_zst_and_save(file_path, output_file):
    """
    Processes a .zst file and saves the valid data to a CSV file.
    """
    selected_data, _ = read_and_process_comment_zst(file_path)
    if selected_data is not None:
        selected_data.to_csv(output_file, index=False)
        print(f'Data saved to {output_file}')
    else:
        print(f"Failed to process {file_path}")

# ============================================================
# Main Code Execution
# ============================================================

# Initial Data Processing for 'SingaporeRaw_comments.zst'
fp_raw_com = 'SingaporeRaw_comments.zst'
df_raw_com = read_and_decode_zst(fp_raw_com)

# Display columns and earliest date
print("Columns in the raw data:")
print(df_raw_com.columns.tolist())

earliest_timestamp = df_raw_com.created_utc.min()
print(f"Earliest created_utc timestamp: {earliest_timestamp}")

# Convert timestamp to human-readable format
earliest_date = datetime.utcfromtimestamp(earliest_timestamp)
print(f"Earliest date (UTC): {earliest_date.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# Sanity check: length of DataFrame
print(f"Length of the raw DataFrame: {len(df_raw_com)}")

# Filter and save columns of interest
df_raw = df_raw_com[['author', 'author_created_utc', 'body', 'created_utc', 'id', 
                     'link_id', 'parent_id', 'score', 'retrieved_utc', 'subreddit']]
df_raw.rename(columns={'score': 'votes'}, inplace=True)
df_raw.to_csv('raw_com.csv', index=False)

# Process additional files
file_paths = {
    'SingaporeRaw_comments.zst': 'raw_com.csv',
    'singapore_comments.zst': 'sg_com.csv',
    'taiwan_comments.zst': 'tw_com.csv',
    'HongKong_comments.zst': 'hk_com.csv',
}

for file_path, output_file in file_paths.items():
    print(f"Processing {file_path}...")
    process_zst_and_save(file_path, output_file)

# ============================================================
# Acknowledgements
# ============================================================
# Thanks to Emmanuel Djanga for troubleshooting and suggestions.


## For Reddit Submission

In [ ]:
import json
import io
import zstandard
import pandas as pd
from datetime import datetime

# ============================================================
# Function Definitions
# ============================================================

def read_and_decode_zst(file_path):
    """
    Reads and decompresses a .zst file, then decodes the JSON data into a pandas DataFrame.
    """
    with open(file_path, 'rb') as file_handle:
        reader = zstandard.ZstdDecompressor().stream_reader(file_handle)
        data = reader.read().decode()
        data = pd.read_json(io.StringIO(data), lines=True)
    return data

def read_and_process_submission_zst(file_path, min_timestamp=1551843493):
    """
    Reads a .zst file line by line, extracts relevant data, and returns two DataFrames:
    one with valid data and one with errors.
    """
    selected_data = []
    error_data = []

    try:
        with open(file_path, 'rb') as file_handle:
            reader = zstandard.ZstdDecompressor().stream_reader(file_handle)
            text_reader = io.TextIOWrapper(reader, encoding='utf-8')
            for line in text_reader:
                try:
                    data_dict = json.loads(line)
                    created_utc = int(data_dict.get('created_utc', 0))
                    if created_utc >= min_timestamp:
                        # Extract relevant fields
                        extracted_data = {
                            'author': data_dict.get('author'),
                            'author_created_utc': data_dict.get('author_created_utc'),
                            'body_title': data_dict.get('title', ''),
                            'body_text': data_dict.get('selftext', ''),
                            'created_utc': data_dict.get('created_utc'),
                            'id': data_dict.get('id'),
                            'votes': data_dict.get('score'),
                            'retrieved_utc': data_dict.get('retrieved_on'),
                            'subreddit': data_dict.get('subreddit'),
                        }
                        # Concatenate body_title and body_text into 'body'
                        extracted_data['body'] = f"{extracted_data['body_title']} {extracted_data['body_text']}".strip()
                        selected_data.append(extracted_data)
                except json.JSONDecodeError as json_err:
                    error_data.append({'error': f'JSON decoding error: {json_err}', 'data': line})
                except Exception as e:
                    error_data.append({'error': str(e), 'data': line})
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
        return None, None

    df_selected_data = pd.DataFrame(selected_data)
    df_error_data = pd.DataFrame(error_data)

    print(f'Based on the modified method, the earliest comment in the data is UTC {df_selected_data.created_utc.min()}') 
    print(f'The selected data has {len(df_selected_data)} rows.')
    print(f'The data with errors has {len(df_error_data)} rows.')

    return df_selected_data, df_error_data

# ============================================================
# Main Code Execution
# ============================================================

# Process the raw submissions to find the earliest date and columns of interest
fp_raw_sub = 'SingaporeRaw_submissions.zst'
df_raw_sub = read_and_decode_zst(fp_raw_sub)

# Display column names
print("Columns in the raw data:")
print(df_raw_sub.columns.tolist())

# Keep columns of interest and export to CSV
df_raw_sub = df_raw_sub[['author', 'author_created_utc', 'title', 'selftext',
                         'created_utc', 'id', 'score', 'retrieved_on', 'subreddit']]
df_raw_sub.rename(columns={'score': 'votes'}, inplace=True)
df_raw_sub.to_csv('raw_sub.csv', index=False)

# Print earliest created date
earliest_timestamp = df_raw_sub.created_utc.min()
print(f"Earliest created_utc timestamp: {earliest_timestamp}")

# Convert timestamp to human-readable format
earliest_date = datetime.utcfromtimestamp(earliest_timestamp)
print(f"Earliest date (UTC): {earliest_date.strftime('%Y-%m-%d %H:%M:%S UTC')}")

# Process additional submission files
datasets = [
    ('SingaporeRaw_submissions.zst', 'raw_sub.csv'),
    ('singapore_submissions.zst', 'sg_sub.csv'),
    ('taiwan_submissions.zst', 'tw_sub.csv'),
    ('HongKong_submissions.zst', 'hk_sub.csv'),
]

for input_file, output_file in datasets:
    print(f"Processing {input_file}...")
    selected_data, _ = read_and_process_submission_zst(input_file)
    if selected_data is not None:
        selected_data.to_csv(output_file, index=False)
        print(f'Data saved to {output_file}')
    else:
        print(f"Failed to process {input_file}")

# ============================================================
# Acknowledgements
# ============================================================
# Thanks to Emmanuel Djanga for troubleshooting and suggestions.


# Import Data and Text Pre-Processing

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import html
import emoji
import re
from datetime import datetime
from urllib.parse import unquote

# Function to extract URLs
def extract_url(df):
    df['extract_url'] = df['body'].apply(extract_url_from_text)
    return df

def extract_url_from_text(text):
    if pd.notna(text):
        soup = BeautifulSoup(text, 'html.parser')
        urls = [link.get('href') for link in soup.find_all('a')]
        return urls
    else:
        return []

# Function to parse URLs
def parse_url(df):
    df['extract_url'] = df['body'].apply(parse_url_from_text)
    return df

def parse_url_from_text(text):
    urls = extract_url_from_text(text)
    decoded_urls = [unquote(url) if isinstance(url, str) else '' for url in urls]
    return decoded_urls

# Function to clean text
def clean_text(df):
    df['cleaned_body'] = df['body'].apply(clean_text_from_text)
    return df

def clean_text_from_text(text):
    if isinstance(text, str):
        cleaned_text = html.unescape(text)
        soup = BeautifulSoup(cleaned_text, 'html.parser')
        cleaned_text = soup.get_text(separator='')
        cleaned_text = emoji.demojize(cleaned_text)
        cleaned_text = re.sub(r'[^\w\s]', '', cleaned_text)
        cleaned_text = cleaned_text.replace('\n', ' ')
        cleaned_text = cleaned_text.strip()
        return cleaned_text
    else:
        return text

# Function to calculate account age
def account_age(df):
    df['account_age'] = df['author_created_utc'].apply(account_age_days)
    return df

def account_age_days(epoch_timestamp):
    current_timestamp = datetime.utcnow().timestamp()
    account_age_seconds = current_timestamp - epoch_timestamp
    account_age_days = account_age_seconds / (24 * 60 * 60)
    return round(account_age_days, 2)

# Function to count words
def count_words(df):
    df['raw_wc'] = df['body'].apply(count_words_from_text)
    df['proc_wc'] = df['cleaned_body'].apply(count_words_from_text)
    return df

def count_words_from_text(text):
    if isinstance(text, str):
        cleaned_text = re.sub(r'[^\w\s]', '', text)
        word_count = len(cleaned_text.split())
        return word_count
    else:
        return 0

# Function to print deletion comments
def print_deleted_comments_stats(df, subreddit):
    delete_comments = (df['body'] == '[deleted]').sum()
    delete_authors = (df['author'] == '[deleted]').sum()
    delete_comments_percent = delete_comments / len(df) * 100
    delete_authors_percent = delete_authors / len(df) * 100
    
    print(subreddit)
    print("Number of '[deleted]' comments:", delete_comments)
    print("Number of '[deleted]' authors:", delete_authors)
    print(f"Percentage of '[delete]' comments: {delete_comments_percent:.2f}%")
    print(f"Percentage of '[delete]' authors: {delete_authors_percent:.2f}%\n")

# Function to print deletion submission
def print_deleted_submission_stats(df, subreddit):
    delete_submission = (df['body_text'] == '[deleted]').sum()
    delete_authors = (df['author'] == '[deleted]').sum()
    delete_submission_percent = delete_submission / len(df) * 100
    delete_authors_percent = delete_authors / len(df) * 100
    
    print(subreddit)
    print("Number of '[deleted]' submission:", delete_submission)
    print("Number of '[deleted]' authors:", delete_authors)
    print(f"Percentage of '[delete]' comments: {delete_submission_percent:.2f}%")
    print(f"Percentage of '[delete]' authors: {delete_authors_percent:.2f}%\n")

def filter_and_save_deleted_comments(df, filename):
    # Filter rows where 'body' or 'author' is '[deleted]'
    deleted_rows = df[(df['body'] == '[deleted]') | (df['author'] == '[deleted]')]
    
    # Save the deleted rows to a new CSV file
    original_filename = filename.split('.')[0]
    deleted_filename = f'{original_filename}_deleted.csv'
    deleted_rows.to_csv(deleted_filename, index=False)
    
    # Return the dataframe without the deleted rows
    return df[(df['body'] != '[deleted]') & (df['author'] != '[deleted]')]

def filter_and_save_deleted_submissions(df, filename):
    # Filter rows where 'body_text' or 'author' is '[deleted]'
    deleted_rows = df[(df['body_text'] == '[deleted]') | (df['author'] == '[deleted]')]
    
    # Save the deleted rows to a new CSV file
    original_filename = filename.split('.')[0]
    deleted_filename = f'{original_filename}_deleted.csv'
    deleted_rows.to_csv(deleted_filename, index=False)
    
    # Return the dataframe without the deleted rows
    return df[(df['body_text'] != '[deleted]') & (df['author'] != '[deleted]')]

# Function to export processed dataframe to CSV with a suffix added to the original filename
def export_to_csv(df, filename):
    # Extract the original filename without the extension
    original_filename = filename.split('.')[0]
    # Construct the new filename with the "_proc.csv" suffix
    new_filename = f'{original_filename}_proc.csv'
    # Export dataframe to CSV
    df.to_csv(new_filename, index=False)

# Table 1 - Reddit Comments with Deleted Author / Comment

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup  # For parsing HTML and extracting URLs
import html  # For HTML unescaping
import emoji  # For handling emojis
import re  # For regular expressions
from datetime import datetime  # For handling date and time

# Load datasets
datasets = ['raw_com.csv', 'sg_com.csv', 'tw_com.csv', 'hk_com.csv', 'omc_com.csv', 'tomc_com.csv']

for dataset in datasets:
    df = pd.read_csv(dataset)
    
    # Process each dataframe
    df = extract_url(df)
    df = parse_url(df)
    df = clean_text(df)
    df = account_age(df)
    df = count_words(df)
    
    # Print deletion statistics
    print_deleted_comments_stats(df, dataset)

    # Filter and save deleted rows 
    df = filter_and_save_deleted_comments(df, dataset)
    
    # Filter out rows with empty 'cleaned_body'
    df = df[df['proc_wc'] > 0]
    
    # Print total number of rows after filtering
    print(f'Total Number of Rows (After Dropping Empty Comments) for {dataset}: {len(df)}')
    
    # Export processed dataframe to CSV with "_proc.csv" suffix added to original filename
    export_to_csv(df, dataset)

# Table 2 - Reddit Submission with Deleted Author / Comment

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import html
import emoji
import re
from datetime import datetime

# ****************************************************************************
# Note the output for the following cell should be submission and not comments
# Was reflected as comments because the function was written for comments, but the underlying logic is similiar
# Load datasets
datasets = ['raw_sub.csv', 'sg_sub.csv', 'tw_sub.csv', 'hk_sub.csv', 'omc_sub.csv', 'tomc_sub.csv']

for dataset in datasets:
    df = pd.read_csv(dataset)
    
    # Process each dataframe
    df = extract_url(df)
    df = parse_url(df)
    df = clean_text(df)
    df = account_age(df)
    df = count_words(df)
    
    # Print deletion statistics
    print_deleted_submission_stats(df, dataset)

    # Filter and save deleted rows 
    df = filter_and_save_deleted_submissions(df, dataset)
    
    # Filter out rows with empty 'cleaned_body'
    df = df[df['proc_wc'] > 0]
    
    # Print total number of rows after filtering
    print(f'Total Number of Rows (After Dropping Empty Submissions) for {dataset}: {len(df)}')
    
    # Export processed dataframe to CSV with "_proc.csv" suffix added to original filename
    export_to_csv(df, dataset)

# Features Extraction

## Checkpoint 1: Incivility

In [ ]:
import pandas as pd
import nltk
from nltk.stem import SnowballStemmer

# ============================================================
# Initialization
# ============================================================

# Initialize Snowball stemmer for English
snowball_stemmer = SnowballStemmer(language='english')

# List of word/word stems to check for
word_stems = [
    "dumb", "farc", "hypocrit", "insan", "insecur", "irrespons", "sham", "trivial", "unaccept", "uneth", "unfair",
    "bigot", "bigotri", "discrimin", "prejudic", "racism", "racist", "segreg", "stereotyp", "betray", "enemi", 
    "insurg", "overthrow", "riot", "threat", "threaten", "traitor", "treason", "tyranni", "brainwash", "deceiv", 
    "decept", "dishonest", "disingenu", "inaccur", "incorrect", "misinform", "mislead", "uninform", "dysfunct", 
    "infring", "obstruct", "obstructionist", "suppress", "unconstitut", "chao", "debacl", "delud", "demean", 
    "disrespect", "fiasco", "inappropri", "nasti", "vitriol", "yell"
]

def calculate_proportion_of_word_stems(text):
    """
    Calculate the proportion of words in the text that match the specified word stems.
    """
    if not isinstance(text, str):
        return 0
    
    # Convert text to lowercase and tokenize
    text = text.lower()
    tokens = nltk.word_tokenize(text)
    
    # Stem the tokens
    stemmed_tokens = [snowball_stemmer.stem(token) for token in tokens]
    
    # Calculate the proportion of stemmed tokens that match the word stems
    word_count = len(tokens)
    count_matches = sum(1 for token in stemmed_tokens if token in word_stems)
    
    return (count_matches / word_count * 100) if word_count > 0 else 0

def process_datasets(datasets):
    """
    Process each dataset to calculate the proportion of word stems in the 'cleaned_body' column.
    """
    for dataset in datasets:
        df = pd.read_csv(dataset)
        df['incivility_prop'] = df['cleaned_body'].apply(calculate_proportion_of_word_stems)
        df.to_csv(dataset, index=False)
        print(f"Processed {dataset}")

# ============================================================
# Define File List
# ============================================================

word_stem_datasets = [
    'raw_sub_proc.csv', 'sg_sub_proc.csv', 'tw_sub_proc.csv', 'hk_sub_proc.csv',
    'raw_com_proc.csv', 'sg_com_proc.csv', 'tw_com_proc.csv', 'hk_com_proc.csv'
]

# ============================================================
# Main Execution
# ============================================================

process_datasets(word_stem_datasets)

## Checkpoint 1.5: Assessing the Impact of Accounting/Not Accounting for Chinese Keywords

In [ ]:
import pandas as pd
import re

def contains_chinese(text):
    """
    Check if the given text contains Chinese characters.
    """
    if pd.isna(text):  # Check for NaN values
        return False
    return bool(re.search(r'[\u4e00-\u9fff]', text))

def analyze_chinese_characters(file_names, text_column, word_count_column, min_word_count):
    """
    Analyze the percentage of rows containing Chinese characters before and after applying a word count filter.
    """
    for file_name in file_names:
        df = pd.read_csv(file_name)
        
        if text_column in df.columns and word_count_column in df.columns:
            df['contains_chinese'] = df[text_column].apply(contains_chinese)
            
            # Calculate percentage of rows containing Chinese characters before filtering
            percentage_before = df['contains_chinese'].mean() * 100
            
            # Apply the word count filter
            filtered_df = df[df[word_count_column] > min_word_count]
            
            # Calculate percentage of rows containing Chinese characters after filtering
            percentage_after = filtered_df['contains_chinese'].mean() * 100 if len(filtered_df) > 0 else 0.0
            
            print(f'File: {file_name}')
            print(f'Percentage of rows containing Chinese characters (before filter): {percentage_before:.2f}%')
            print(f'Percentage of rows containing Chinese characters (after filter): {percentage_after:.2f}%')
        else:
            print(f'Column "{text_column}" or "{word_count_column}" not found in file: {file_name}')

# ============================================================
# Define File List and Parameters
# ============================================================

chinese_char_datasets = [
    'raw_com_proc.csv',
    'sg_com_proc.csv',
    'tw_com_proc.csv',
    'hk_com_proc.csv'
]

# Column names for processing
text_column_name = 'cleaned_body'
word_count_column_name = 'proc_wc'
min_word_count = 9  # Minimum word count to apply the filter

# ============================================================
# Main Execution
# ============================================================

analyze_chinese_characters(chinese_char_datasets, text_column_name, word_count_column_name, min_word_count)

## Checkpoint 2: Queer vs. Non-Queer

In [ ]:
import pandas as pd
import re
from fuzzywuzzy import process, fuzz

# Datasets
datasets = {
    'raw_com': 'raw_com_proc.csv',
    'sg_com': 'sg_com_proc.csv',
    'tw_com': 'tw_com_proc.csv',
    'hk_com': 'hk_com_proc.csv',
    'omc_com': 'omc_com_proc.csv',
    'tomc_com': 'tomc_com_proc.csv',
    'raw_sub': 'raw_sub_proc.csv',
    'sg_sub': 'sg_sub_proc.csv',
    'tw_sub': 'tw_sub_proc.csv',
    'hk_sub': 'hk_sub_proc.csv',
    'omc_sub': 'omc_sub_proc.csv',
    'tomc_sub': 'tomc_sub_proc.csv'
}

# Predefined keywords
predefined_keywords = [
    'gay', 'lesbian', 'lgbt', 'queer', 'homosexual', 'faggot', 'sissy', 'tranny', 'ahkua', 'ahgua'
]

# Mixed characters list for matching
mixed_characters_list = [
    '同性戀', '同性恋', '同志', '娘娘腔', '娘炮', '基佬', 'ah kua', 'ah gua'
]

# Case-insensitive regex pattern
pattern = re.compile('|'.join(mixed_characters_list), re.IGNORECASE)

## Helper Functions 

#Function to Find Keyword Matches

def find_matches(text, keywords, threshold=90):
    """Find fuzzy matches of keywords within the text."""
    text = text.lower()
    matches = process.extract(text, keywords, scorer=fuzz.partial_ratio)
    matched_keywords = [match[0] for match in matches if match[1] >= threshold]
    return matched_keywords

#Function to Process a DataFrame

def process_dataframe(df):
    """Process a DataFrame to detect keywords and compute related metrics."""
    # Ensure 'cleaned_body' column is of type string
    df['cleaned_body'] = df['cleaned_body'].astype(str)

    # Apply matching functions
    df['matched_keywords'] = df['cleaned_body'].apply(lambda x: find_matches(x, predefined_keywords))
    df['matched_keywords_mix'] = df['cleaned_body'].str.lower().str.findall(pattern)

    # Create binary columns for keyword presence
    df['keywords_present_eng'] = df['matched_keywords'].apply(lambda x: 1 if x else 0)
    df['keywords_present_mix'] = df['matched_keywords_mix'].apply(lambda x: 1 if x else 0)
    df['keywords_present_all'] = df.apply(
        lambda row: 1 if (row['keywords_present_eng'] == 1 or row['keywords_present_mix'] == 1) else 0, axis=1
    )

    # Calculate and print the proportion of rows with LGBTQ-related content
    proportion = len(df[df['keywords_present_all'] == 1]) / len(df) * 100
    print(f"Proportion with LGBTQ-related content: {proportion:.3f}%")
    
    return df

In [ ]:
# Processing the Datasets

def process_and_save_datasets(datasets):
    """Process each dataset and save the results to a new CSV."""
    for name, filename in datasets.items():
        df = pd.read_csv(filename)
        df = process_dataframe(df)
        df.to_csv(f'{name}_proc.csv', index=False)

process_and_save_datasets(datasets)

In [ ]:
# Statistical Analysis - Compute Binary Statistics

def compute_binary_stats(file_path, dataset_name):
    """Compute and print statistics for the 'keywords_present_all' binary column."""
    df = pd.read_csv(file_path)
    
    if 'keywords_present_all' in df.columns:
        count_ones = df['keywords_present_all'].sum()
        count_zeros = len(df) - count_ones
        mean_value = df['keywords_present_all'].mean()
        std_dev_value = df['keywords_present_all'].std()
        
        print(f"Dataset: {dataset_name}")
        print(f"Count of '1's: {count_ones}")
        print(f"Count of '0's: {count_zeros}")
        print(f"Mean of 'keywords_present_all': {mean_value:.3f}")
        print(f"Standard Deviation of 'keywords_present_all': {std_dev_value:.3f}\n")
    else:
        print(f"Dataset: {dataset_name} does not contain 'keywords_present_all' column.\n")

# Iterate through the datasets and compute metrics for each dataset
for key, file in datasets.items():
    compute_binary_stats(file, key)

In [ ]:
# Statistics Before and After Filtering for Word Count

def compare_before_after_filter(csv_files, min_word_count=9):
    """Compare the difference in keyword presence before and after filtering."""
    for file in csv_files:
        df = pd.read_csv(file)
        
        # Before filtering
        percentage_eng_before = df['keywords_present_eng'].mean() * 100
        percentage_all_before = df['keywords_present_all'].mean() * 100
        difference_before = percentage_all_before - percentage_eng_before
        
        # Apply filter
        filtered_df = df[df['proc_wc'] > min_word_count]
        
        if not filtered_df.empty:
            # After filtering
            percentage_eng_after = filtered_df['keywords_present_eng'].mean() * 100
            percentage_all_after = filtered_df['keywords_present_all'].mean() * 100
            difference_after = percentage_all_after - percentage_eng_after
        else:
            # No data after filtering
            percentage_eng_after, percentage_all_after, difference_after = 0, 0, 0
        
        # Print results
        print(f"File: {file}")
        print(f"Difference before filtering: {difference_before:.2f}%")
        print(f"Difference after filtering: {difference_after:.2f}%\n")

# List of CSV files
csv_files = ['raw_com_proc.csv', 'sg_com_proc.csv', 'tw_com_proc.csv', 'hk_com_proc.csv']

compare_before_after_filter(csv_files)

## Checkpoint 3: Number of Replies (Queer vs. Non-Queer)

In [ ]:
import pandas as pd
import numpy as np

def load_and_merge_dataframes(sub_proc_file, com_proc_file, common_columns):
    """Load and merge the two CSV files based on the common columns."""
    df_sub_proc = pd.read_csv(sub_proc_file)
    df_com_proc = pd.read_csv(com_proc_file)
    
    df_com_proc = df_com_proc[common_columns + ['parent_id']]
    df_sub_proc = df_sub_proc[common_columns]
    
    return pd.concat([df_sub_proc, df_com_proc], ignore_index=True)

def filter_dataframe_by_keyword(df):
    """Filter the dataframe by 'keywords_present_all' and return two filtered dataframes."""
    df_filter_0 = df[df['keywords_present_all'] == 0]
    df_filter_1 = df[df['keywords_present_all'] == 1]
    
    return df_filter_0, df_filter_1

def count_occurrences(parent_ids, ids_list):
    """Count the occurrences of each ID in the parent_id list."""
    counts = {id_: 0 for id_ in ids_list}
    for parent_id in parent_ids:
        parent_id = parent_id.split('_')[1]  # Remove the prefix
        if parent_id in counts:
            counts[parent_id] += 1
    return counts

def compute_statistics(counts_dict):
    """Compute count, average, and standard deviation from the occurrence counts."""
    values = list(counts_dict.values())
    count = len(values)
    average = np.mean(values)
    std_dev = np.std(values)
    return count, average, std_dev

def process_dataset(sub_proc_file, com_proc_file, common_columns):
    """Process the dataset, compute statistics, and return the counts and statistics."""
    df = load_and_merge_dataframes(sub_proc_file, com_proc_file, common_columns)
    df_filter_0, df_filter_1 = filter_dataframe_by_keyword(df)

    id_list_filter_0 = df_filter_0['id'].tolist()
    id_list_filter_1 = df_filter_1['id'].tolist()

    parent_ids = df['parent_id'].dropna().tolist()

    count_filter_0 = count_occurrences(parent_ids, id_list_filter_0)
    count_filter_1 = count_occurrences(parent_ids, id_list_filter_1)

    stats_filter_0 = compute_statistics(count_filter_0)
    stats_filter_1 = compute_statistics(count_filter_1)

    return count_filter_0, stats_filter_0, count_filter_1, stats_filter_1

def save_and_print_results(counts, stats, file_prefix):
    """Save counts to CSV and print statistics."""
    df_counts = pd.DataFrame(list(counts.items()), columns=('id', 'count'))
    df_counts.to_csv(f'{file_prefix}.csv', index=False)
    
    print(f"Counts for {file_prefix}:")
    print(df_counts)
    print(f"Statistics: Count = {stats[0]}, Average = {stats[1]:.2f}, Std Dev = {stats[2]:.2f}")

def process_all_datasets(dataset_pairs, common_columns):
    """Process all dataset pairs and save the results."""
    for name, (sub_proc_file, com_proc_file) in dataset_pairs.items():
        count_filter_0, stats_filter_0, count_filter_1, stats_filter_1 = process_dataset(sub_proc_file, com_proc_file, common_columns)
        
        save_and_print_results(count_filter_0, stats_filter_0, f'{name}_reply_count_nonqueer')
        save_and_print_results(count_filter_1, stats_filter_1, f'{name}_reply_count_queer')

# Define common columns and dataset pairs
common_columns = [
    'author', 'author_created_utc', 'body', 'created_utc', 'id', 'votes', 
    'retrieved_utc', 'subreddit', 'extract_url', 'cleaned_body', 
    'account_age', 'raw_wc', 'proc_wc', 'incivility_prop', 'matched_keywords', 
    'keywords_present_eng', 'matched_keywords_mix', 'keywords_present_mix', 
    'keywords_present_all'
]

dataset_pairs = {
    'raw': ('raw_sub_proc.csv', 'raw_com_proc.csv'),
    'sg': ('sg_sub_proc.csv', 'sg_com_proc.csv'),
    'tw': ('tw_sub_proc.csv', 'tw_com_proc.csv'),
    'hk': ('hk_sub_proc.csv', 'hk_com_proc.csv'),
}

# Process all datasets
process_all_datasets(dataset_pairs, common_columns)


## Checkpoint 4: Number of Votes (Queer vs. Non-Queer)

In [ ]:
import pandas as pd

def get_filenames():
    """Return the list of filenames to process."""
    return [
        'tomc_com_proc.csv', 'tomc_sub_proc.csv',
        'tw_com_proc.csv', 'tw_sub_proc.csv',
        'raw_com_proc.csv', 'raw_sub_proc.csv',
        'omc_com_proc.csv', 'omc_sub_proc.csv',
        'hk_com_proc.csv', 'hk_sub_proc.csv',
        'sg_com_proc.csv', 'sg_sub_proc.csv'
    ]

def get_common_columns():
    """Return the list of common columns to select from the files."""
    return ['id', 'votes', 'subreddit', 'keywords_present_all']

def read_and_concatenate_files(filenames, common_columns):
    """Read multiple CSV files and concatenate them into a single DataFrame."""
    dataframes = [pd.read_csv(file, usecols=common_columns) for file in filenames]
    merged_df = pd.concat(dataframes, ignore_index=True)
    return merged_df

def calculate_statistics(df):
    """Calculate descriptive statistics grouped by subreddit and keyword presence."""
    grouped_stats = df.groupby(['subreddit', 'keywords_present_all'])['votes'].agg(['mean', 'std', 'median'])
    print(grouped_stats)
    return grouped_stats

def export_dataframe(df, output_file):
    """Export the DataFrame to a CSV file without the index."""
    df.to_csv(output_file, index=False)

def main():
    """Main function to execute the data processing and export."""
    filenames = get_filenames()
    common_columns = get_common_columns()
    
    merged_df = read_and_concatenate_files(filenames, common_columns)
    calculate_statistics(merged_df)
    export_dataframe(merged_df, 'df_merged_votes.csv')

if __name__ == '__main__':
    main()

## Checkpoint 5: LIWC - Authentic, Tone, Dic, WC, Prosocial, Polite

In [ ]:
#Data Preparation for Feature Extraction

import pandas as pd

def prepare_data_for_extraction(input_files, output_files):
    """
    Prepare data for feature extraction by selecting specific columns and saving to new files.
    
    Parameters:
    - input_files: List of input CSV filenames.
    - output_files: List of output CSV filenames corresponding to each input file.
    """
    for input_file, output_file in zip(input_files, output_files):
        # Read the CSV file
        df = pd.read_csv(input_file)
        
        # Select only the 'id' and 'cleaned_body' columns
        df_filtered = df[['id', 'cleaned_body']]
        
        # Export to new CSV file
        df_filtered.to_csv(output_file, index=False)

    print("Data preparation complete!")

# List of input and output filenames
input_files = [
    'raw_com_proc.csv',
    'sg_com_proc.csv',
    'tw_com_proc.csv',
    'hk_com_proc.csv'
]

output_files = [
    'raw_com_liwc_a2.csv',
    'sg_com_liwc_a2.csv',
    'tw_com_liwc_a2.csv',
    'hk_com_liwc_a2.csv'
]

# Prepare data for feature extraction
prepare_data_for_extraction(input_files, output_files)

In [ ]:
# Files will be sent to LIWC-2022 for feature extraction.
# Only after this process is complete will the next block of code run.

In [ ]:
import pandas as pd

def analyze_extraction_issues(filenames):
    """
    Analyze the issues in feature extraction related to 'Authentic' and 'Tone' columns.
    
    Parameters:
    - filenames: List of CSV filenames to analyze.
    """
    total_deleted_due_to_authentic = 0
    total_deleted_due_to_both = 0

    for filename in filenames:
        # Load the data into a DataFrame
        df = pd.read_csv(filename)

        print(f"\nProcessing file: {filename}")
        
        # Print total number of comments
        print(f"Total Number of Comments: {len(df)}")

        # Count missing values in the 'Authentic' column
        authentic_missing = df['Authentic'].isna().sum()
        print(f"Comments Failed to Extract for Authenticity: {authentic_missing}")
        total_deleted_due_to_authentic += authentic_missing

        # Count missing values in the 'Tone' column
        tone_missing = df['Tone'].isna().sum()
        print(f"Comments Failed to Extract for Tone: {tone_missing}")

        # Find rows where both 'Authentic' and 'Tone' are NaN
        both_missing = df[df['Authentic'].isna() | df['Tone'].isna()]

        # Print the number of comments where both are missing
        both_missing_count = len(both_missing)
        print(f"Comments Failed to Extract for Both Authenticity & Tone: {both_missing_count}")

        # Update total deleted due to both 'Authentic' and 'Tone'
        total_deleted_due_to_both += both_missing_count

        # Drop rows where both 'Tone' and 'Authentic' are NaN
        df_cleaned = df.dropna(subset=['Authentic', 'Tone'])
        print(f"Number of Comments with both Authenticity and Tone: {len(df_cleaned)}")

        # Number of comments deleted when dropping rows with NaN in both 'Tone' and 'Authentic'
        comments_deleted = len(df) - len(df_cleaned)
        print(f"Number of Comments Deleted when dropping rows with NaN in both Authenticity and Tone: {comments_deleted}")

        # Number of comments left if dropping rows with NaN in 'Authentic' only
        df_auth = df.dropna(subset=['Authentic'])
        print(f"Number of Comments left if only Authenticity is Dropped: {len(df_auth)}")

    # Print the total number of comments deleted due to excluding 'Authentic' across all files
    print(f"\nTotal Number of Comments Deleted Due to Excluding Authenticity Across All Files: {total_deleted_due_to_authentic}")

    # Print the total number of comments deleted due to excluding both 'Authentic' and 'Tone' across all files
    print(f"Total Number of Comments Deleted Due to Excluding Both Authenticity and Tone Across All Files: {total_deleted_due_to_both}")

def merge_liwc_output(proc_file, liwc_file, output_file):
    """
    Merge the main DataFrame with the LIWC output.
    
    Parameters:
    - proc_file: Path to the 'proc' CSV file.
    - liwc_file: Path to the 'liwc' CSV file.
    - output_file: Path where the merged DataFrame will be saved.
    """
    try:
        # Load DataFrames
        df_proc = pd.read_csv(proc_file)
        df_liwc = pd.read_csv(liwc_file)
        
        # Columns to drop from df_proc
        columns_to_drop = ['Authentic', 'Tone', 'prosocial', 'polite', 'Segment', 'WC']
        
        # Drop columns if they exist in df_proc
        existing_columns_to_drop = [col for col in columns_to_drop if col in df_proc.columns]
        if existing_columns_to_drop:
            df_proc = df_proc.drop(columns=existing_columns_to_drop)
        else:
            print(f"Warning: None of the columns {columns_to_drop} found in {proc_file}.")
        
        # Specify columns to bring over from df_liwc
        columns_to_bring = ['id', 'WC', 'Authentic']
        
        # Perform the merge
        df_merged = pd.merge(df_proc, df_liwc[columns_to_bring], on='id', how='left')
        
        # Save the merged DataFrame to CSV
        df_merged.to_csv(output_file, index=False)
        print(f"Successfully processed and saved to {output_file}")
    
    except Exception as e:
        print(f"Error processing files {proc_file} and {liwc_file}: {e}")

def main():
    # Analyze extraction issues for files
    extraction_files = [
        'raw_com_liwc_a2.csv',
        'sg_com_liwc_a2.csv',
        'tw_com_liwc_a2.csv',
        'hk_com_liwc_a2.csv'
    ]
    analyze_extraction_issues(extraction_files)

    # Define file pairs and output files for merging
    file_pairs = [
        ('raw_com_proc.csv', 'raw_com_liwc_a2.csv', 'merged_raw_com_proc.csv'),
        ('sg_com_proc.csv', 'sg_com_liwc_a2.csv', 'merged_sg_com_proc.csv'),
        ('tw_com_proc.csv', 'tw_com_liwc_a2.csv', 'merged_tw_com_proc.csv'),
        ('hk_com_proc.csv', 'hk_com_liwc_a2.csv', 'merged_hk_com_proc.csv')
    ]
    
    # Process each file pair for merging
    for proc_file, liwc_file, output_file in file_pairs:
        merge_liwc_output(proc_file, liwc_file, output_file)

if __name__ == "__main__":
    main()


## Checkpoint 6: Create Additional Reddit Features: 'is_reply' (1 = reply, 0 = top-level comments)

In [ ]:
import pandas as pd
import numpy as np

def add_columns_comments(df):
    """
    Add 'is_reply' columns to the comments DataFrame based on 'parent_id'.
    
    Parameters:
    df (pd.DataFrame): Comments DataFrame to process.
    
    Returns:
    pd.DataFrame: Comments DataFrame with added columns.
    """
    df['is_reply'] = np.where(df['parent_id'].str.startswith('t3'), 0,
                           np.where(df['parent_id'].str.startswith('t1'), 1, np.nan))
    return df

def process_and_export(file_paths):
    """
    Process each comments CSV file, add columns, and export the processed DataFrame as CSV.
    
    Parameters:
    file_paths (list of str): List of file paths for comments to process.
    """
    for file_path in file_paths:
        # Read CSV file into DataFrame
        df = pd.read_csv(file_path)
        
        # Apply function to add columns
        df = add_columns_comments(df)
        
        # Ensure DataFrame has the necessary columns
        common_columns = ['author','id', 'parent_id','cleaned_body', 'votes', 'account_age', 
                          'WC', 'incivility_prop','keywords_present_eng', 'matched_keywords_mix',
                          'keywords_present_all','Authentic','is_reply','subreddit']
        
        df = df[common_columns]
        
        # Export DataFrame to CSV with the same name (overwriting)
        df.to_csv(file_path, index=False)
        print(f"Processed and exported: {file_path}")

# File paths to process
file_paths_comments = [
    'merged_hk_com_proc.csv',
    'merged_tw_com_proc.csv',
    'merged_sg_com_proc.csv',
    'merged_raw_com_proc.csv'
]

process_and_export(file_paths_comments)

## Checkpoint 7 : Get number of replies, average emotion in replies, average incivility in replies, keeping enteries with more than 9 wordcount in this processing 

In [ ]:
import pandas as pd

# Define functions

def count_replies(df):
    """
    Counts the number of replies for each 'id' and adds this information to the dataframe.
    """
    df['parent_id_clean'] = df['parent_id'].str.replace(r'^t[13]_', '', regex=True)
    id_counts = df['parent_id_clean'].value_counts().to_dict()
    df['num_of_replies'] = df['id'].map(id_counts).fillna(0).astype(int)
    return df

def merge_incivility(df):
    """
    Calculates the average incivility for each 'parent_id', merges this information
    back into the original dataframe, and fills missing values.
    """
    df_copy = df.copy(deep=True)
    result = df_copy.groupby('parent_id').agg({'incivility_prop': 'mean'}).reset_index()
    result['parent_id'] = result['parent_id'].str.replace(r'^t[13]_', '', regex=True)
    result = result.rename(columns={'parent_id': 'id', 'incivility_prop': 'reply_avg_incivility'})
    df_merged = pd.merge(df, result, on='id', how='left')
    df_merged['reply_avg_incivility'] = df_merged['reply_avg_incivility'].fillna(0)
    return df_merged

def process_csv_files(input_files, output_files, apply_function, filter_condition=None):
    """
    Processes CSV files: reads, applies a function, optionally filters, and saves the results.
    """
    for input_file, output_file in zip(input_files, output_files):
        # Read the CSV file into a DataFrame
        df = pd.read_csv(input_file)
        
        # Apply the specified function
        df = apply_function(df)
        
        # Apply filter condition if specified
        if filter_condition:
            df = df.query(filter_condition)
        
        # Save the modified DataFrame back to a new CSV file
        df.to_csv(output_file, index=False)
        
    print(f"Processing complete. Modified files saved with the suffix '{output_file.split('_')[-1]}'.")

# Step 1: Get number of replies
input_files_step1 = [
    'merged_hk_com_proc.csv',
    'merged_tw_com_proc.csv',
    'merged_sg_com_proc.csv',
    'merged_raw_com_proc.csv'
]

output_files_step1 = [
    'merged_hk_com_proc_with_replies.csv',
    'merged_tw_com_proc_with_replies.csv',
    'merged_sg_com_proc_with_replies.csv',
    'merged_raw_com_proc_with_replies.csv'
]

process_csv_files(input_files_step1, output_files_step1, count_replies)

# Step 2: Average the replies of incivility, regardless of word count
input_files_step2 = [
    'merged_hk_com_proc_with_replies.csv',
    'merged_tw_com_proc_with_replies.csv',
    'merged_sg_com_proc_with_replies.csv',
    'merged_raw_com_proc_with_replies.csv'
]

output_files_step2 = [
    'hk_com_proc_with_replies.csv',
    'tw_com_proc_with_replies.csv',
    'sg_com_proc_with_replies.csv',
    'raw_com_proc_with_replies.csv'
]

process_csv_files(input_files_step2, output_files_step2, merge_incivility, filter_condition='WC > 9')

# Additional Processing: Create final dataset with dummy variables
file_paths_final = [
    'hk_com_proc_with_replies.csv',
    'tw_com_proc_with_replies.csv',
    'sg_com_proc_with_replies.csv',
    'raw_com_proc_with_replies.csv'
]

# Columns to keep
columns_of_interest = ['subreddit', 'keywords_present_all', 'Authentic', 'WC', 'is_reply', 'reply_avg_incivility', 'votes', 'num_of_replies']

# Read and concatenate DataFrames
dfs = [pd.read_csv(file)[columns_of_interest] for file in file_paths_final]
concatenated_df = pd.concat(dfs, ignore_index=True)

# Create dummy variables
dummies = pd.get_dummies(concatenated_df['subreddit'], prefix='subreddit', drop_first=False)
dummies.drop('subreddit_SingaporeRaw', axis=1, inplace=True)

# Concatenate dummy variables with the original DataFrame
df_final = pd.concat([concatenated_df, dummies], axis=1)
df_final[dummies.columns] = df_final[dummies.columns].astype(int)

# Export the final DataFrame
final_output_file = 'final_dataset.csv'
df_final.to_csv(final_output_file, index=False)

print("Final dataset with dummy variables saved as 'final_dataset.csv'.")

# Descriptive Analyses

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

# List of CSV file paths
csv_files = [
    'hk_com_proc_with_replies.csv',
    'tw_com_proc_with_replies.csv',
    'sg_com_proc_with_replies.csv',
    'raw_com_proc_with_replies.csv'
]

def read_and_concatenate_files(file_paths):
    """
    Reads and concatenates multiple CSV files into a single DataFrame.
    """
    return pd.concat((pd.read_csv(file) for file in file_paths), ignore_index=True)

def calculate_stats(dataframe, columns):
    """
    Calculates statistical measures for specified columns in a DataFrame.
    """
    stats = {}
    for column in columns:
        stats[column] = {
            'mean': round(dataframe[column].mean(), 2),
            'std_dev': round(dataframe[column].std(), 2),
            'min': round(dataframe[column].min(), 2),
            'max': round(dataframe[column].max(), 2),
            'skewness': round(skew(dataframe[column].dropna()), 2),
            'kurtosis': round(kurtosis(dataframe[column].dropna()), 2)
        }
    return stats

def print_statistics(stats, title):
    """
    Prints statistical measures for each column in a dictionary.
    """
    print(f"\n{title}:")
    for column, column_stats in stats.items():
        print(f"\n{column}:")
        for stat, value in column_stats.items():
            print(f"{stat}: {value}")

# Main script
df = read_and_concatenate_files(csv_files)

# Print overall DataFrame statistics
print(len(df))
print("\nSubreddit distribution:")
print(df['subreddit'].value_counts())

# Define columns for which statistics are needed
columns = ['Authentic', 'WC', 'reply_avg_tone', 'reply_avg_incivility', 'num_of_replies', 'votes']

# Get and print overall statistics
overall_stats = calculate_stats(df, columns)
print_statistics(overall_stats, "Overall Statistics")

# Filter DataFrame and print statistics for each condition
for keyword_value in [1, 0]:
    df_filtered = df[df['keywords_present_all'] == keyword_value]
    stats = calculate_stats(df_filtered, columns)
    print_statistics(stats, f"Statistics for keywords_present_all == '{keyword_value}'")
    print(f"Number of records: {len(df_filtered)}")

# Appendix A

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator

# Define the plot_data_loss function
def plot_data_loss(df, column, thresholds, file_prefix):
    """
    Plot percentage of data lost for different thresholds on a column in a Pandas DataFrame.

    Parameters:
    - df: Pandas DataFrame
    - column: Name of the column in df to evaluate
    - thresholds: List of values to set as the threshold for column 'WC'
    - file_prefix: Prefix from the file name to include in the plot title

    Returns:
    - None (plots the graph directly)
    """
    total_rows = len(df)
    threshold_values = []
    percentage_lost = []
    
    for threshold in thresholds:
        df_temp = df.copy()
        df_temp.loc[df_temp[column] < threshold, column] = None
        
        rows_lost = df_temp[column].isnull().sum()
        percentage = (rows_lost / total_rows) * 100
        
        threshold_values.append(threshold)
        percentage_lost.append(percentage)
    
    plt.figure(figsize=(10, 6))
    plt.plot(threshold_values, percentage_lost, marker='o')
    plt.title(f'Percentage of Data Lost Due to Minimum Word Count Threshold: {file_prefix}')
    plt.xlabel('Minimum Word Count Threshold')
    plt.ylabel('Percentage of Data Lost')
    plt.grid(True)

    # Set the x-axis major ticks to be every 2 units
    plt.gca().xaxis.set_major_locator(MultipleLocator(2))  # Major ticks on x-axis every 2 units
    
    # Set the y-axis major ticks to be every 10 units (or use default if preferred)
    plt.gca().yaxis.set_major_locator(MultipleLocator(10))  # Major ticks on y-axis every 10 units
    
    plt.show()

# List of file paths
file_paths = [
    'hk_com_proc.csv',
    'tw_com_proc.csv',
    'sg_com_proc.csv',
    'raw_com_proc.csv'
]

# List of thresholds to test
thresholds = [2, 4, 6, 8, 10, 12, 14, 16, 18, 20]


# Iterate through each file path
for file_path in file_paths:
    print(f"Processing file: {file_path}")

    # Extract the file prefix for use in the plot title
    file_prefix = file_path.split('_')[0]
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Assuming 'WC' is the column of interest
    plot_data_loss(df, 'WC', thresholds, file_prefix)

In [ ]:
import pandas as pd

# Define a function to calculate the exact percentage of data lost and the number of lost entries for a given threshold on a column in a Pandas DataFrame
def calculate_data_loss_at_threshold(df, column, threshold):
    """
    Calculate the exact number of data lost and the percentage of data lost for a given threshold on a column in a Pandas DataFrame.
    
    Parameters:
    - df: Pandas DataFrame
    - column: Name of the column in df to evaluate
    - threshold: Threshold value to set for column 'WC'
    
    Returns:
    - A tuple containing the absolute number of entries lost and the percentage of data lost at the given threshold
    """
    total_rows = len(df)
    df_temp = df.copy()
    df_temp.loc[df_temp[column] < threshold, column] = None
    
    rows_lost = df_temp[column].isnull().sum()
    percentage_lost = (rows_lost / total_rows) * 100
    
    return rows_lost, percentage_lost

# List of file paths
file_paths = [
    'hk_com_proc.csv',
    'tw_com_proc.csv',
    'sg_com_proc.csv',
    'raw_com_proc.csv'
]

# Define the specific threshold value
specific_threshold = 10

# Iterate through each file path
for file_path in file_paths:
    print(f"Processing file: {file_path}")
    
    # Load the CSV file into a DataFrame
    df = pd.read_csv(file_path)
    
    # Calculate and get the absolute number of entries lost and the percentage of data lost at the threshold of 10
    rows_lost, percentage_lost_at_threshold = calculate_data_loss_at_threshold(df, 'WC', specific_threshold)
    
    # Print the results
    print(f"Absolute number of entries lost at threshold {specific_threshold} for {file_path}: {rows_lost}")
    print(f"Percentage of data lost at threshold {specific_threshold} for {file_path}: {percentage_lost_at_threshold:.2f}%")
    print()  # Print a blank line for better readability between files

# Logistics Regression (Inference) 
Purpose: Replicate the Results in R and Obtain Linearity Plots

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns

# Define file path
file_path = 'final_dataset.csv'

def load_and_preprocess_data(file_path):
    """
    Load dataset and drop rows with missing values in relevant columns.
    """
    df = pd.read_csv(file_path)
    df = df.dropna(subset=[
        'subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
        'Authentic', 'WC', 'is_reply', 'keywords_present_all'
    ])
    return df

def fit_logistic_regression(X, y):
    """
    Fit a logistic regression model and return the fitted model.
    """
    X = sm.add_constant(X)
    model = sm.Logit(y, X).fit()
    return model

def print_model_summary(model):
    """
    Print the summary of a fitted logistic regression model.
    """
    print(model.summary())

def print_model_metrics(model):
    """
    Print coefficients, AIC, and BIC of a fitted logistic regression model.
    """
    coefficients = model.params
    print("Coefficients:")
    for variable, coef in coefficients.items():
        print(f"{variable}: {coef:.2f}")

    aic = model.aic
    bic = model.bic
    print(f"AIC: {round(aic, 2)}")
    print(f"BIC: {round(bic, 2)}")

def plot_log_odds(X, coefficients, predictors):
    """
    Plot log odds against each predictor.
    """
    log_odds = X.dot(coefficients)
    
    plt.figure(figsize=(18, 10))
    colors = ['blue', 'green', 'red', 'purple', 'orange']
    
    for i, predictor in enumerate(predictors):
        plt.subplot(2, 3, i + 1)
        if predictor in X.columns:
            plt.scatter(X[predictor], log_odds, color=colors[i], alpha=0.5)
            plt.xlabel(predictor)
            plt.ylabel('Log Odds')
            plt.title(f'Log Odds vs. {predictor}')
            plt.grid(True)
    
    plt.tight_layout()
    plt.show()

def nlog(score, epsilon=1e-6):
    """
    Apply log transformation to handle zero values.
    """
    return np.sign(score) * np.log(np.abs(score) + epsilon)

# Model M1: Reddit Source - No Log Transformation
df = load_and_preprocess_data(file_path)
X1 = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan']]
y1 = df['keywords_present_all']
model_1 = fit_logistic_regression(X1, y1)
print("Model 1:")
print_model_summary(model_1)
print_model_metrics(model_1)

# Model M2: Reddit Source + User-Generated Features (Before Logging)
X2 = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
        'Authentic', 'WC', 'is_reply']]
y2 = df['keywords_present_all']
model_2 = fit_logistic_regression(X2, y2)
print("\nModel 2:")
print_model_summary(model_2)
print_model_metrics(model_2)
plot_log_odds(X2, model_2.params, ['Authentic', 'WC'])

# Model M2Log: Reddit Source + User-Generated Features (Logged)
df[['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes']] = df[['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes']].apply(nlog)
X2_log = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
             'Authentic', 'WC', 'is_reply', 'reply_avg_incivility', 'num_of_replies', 'votes']]
y2_log = df['keywords_present_all']
model_2Log = fit_logistic_regression(X2_log, y2_log)
print("\nModel 2 Log:")
print_model_summary(model_2Log)
print_model_metrics(model_2Log)
plot_log_odds(X2_log, model_2Log.params, ['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes'])

# Model M3: Reddit Source + User-Generated Features + Feedback Features (Before Logging)
X3 = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
        'Authentic', 'WC', 'is_reply', 'reply_avg_incivility', 'num_of_replies', 'votes']]
y3 = df['keywords_present_all']
model_3 = fit_logistic_regression(X3, y3)
print("\nModel 3:")
print_model_summary(model_3)
print_model_metrics(model_3)
plot_log_odds(X3, model_3.params, ['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes'])

# Model M3Log: Reddit Source + User-Generated Features + Feedback Features (Logged)
df[['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes']] = df[['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes']].apply(nlog)
X3_log = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
             'Authentic', 'WC', 'is_reply', 'reply_avg_incivility', 'num_of_replies', 'votes']]
y3_log = df['keywords_present_all']
model_3Log = fit_logistic_regression(X3_log, y3_log)
print("\nModel 3 Log:")
print_model_summary(model_3Log)
print_model_metrics(model_3Log)
plot_log_odds(X3_log, model_3Log.params, ['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes'])

# Model M4: Reddit Source + User-Generated Features + Feedback Features (Incivility Not Logged)
df[['Authentic', 'WC', 'num_of_replies', 'votes']] = df[['Authentic', 'WC', 'num_of_replies', 'votes']].apply(nlog)
X4 = df[['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
        'Authentic', 'WC', 'is_reply', 'reply_avg_incivility', 'num_of_replies', 'votes']]
y4 = df['keywords_present_all']
model_4 = fit_logistic_regression(X4, y4)
print("\nModel 4:")
print_model_summary(model_4)
print_model_metrics(model_4)
plot_log_odds(X4, model_4.params, ['Authentic', 'WC', 'reply_avg_incivility', 'num_of_replies', 'votes'])


### Descriptives for Log-Transformed Variables

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import skew, kurtosis

def load_and_transform_data(file_path):
    # Load the dataset
    df = pd.read_csv(file_path)
    
    # Log Transformation
    df['Authentic'] = np.log(df['Authentic'])
    df['WC'] = np.log(df['WC'])
    df['reply_avg_tone'] = np.log(df['reply_avg_tone'])
    df['reply_avg_incivility'] = np.log(df['reply_avg_incivility'] + 0.001)
    df['num_of_replies'] = np.log(df['num_of_replies'] + 0.001)
    df['votes'] = np.log(df['votes'] + 2392.001)
    
    return df

def calculate_stats(dataframe, columns):
    stats = {}
    for column in columns:
        column_data = dataframe[column].dropna()
        stats[column] = {
            'mean': round(column_data.mean(), 2),
            'std_dev': round(column_data.std(), 2),
            'min': round(column_data.min(), 2),
            'max': round(column_data.max(), 2),
            'skewness': round(skew(column_data), 2),
            'kurtosis': round(kurtosis(column_data), 2)
        }
    return stats

def analyze_statistics(df, columns):
    # Overall statistics
    overall_stats = calculate_stats(df, columns)
    print("Overall Statistics:")
    for column, stats in overall_stats.items():
        print(f"\n{column}:")
        for stat, value in stats.items():
            print(f"{stat}: {value}")

    # Statistics for keywords_present_all == 1
    df_keywords_1 = df[df['keywords_present_all'] == 1]
    keywords_1_stats = calculate_stats(df_keywords_1, columns)
    print("\nStatistics for keywords_present_all == '1':")
    for column, stats in keywords_1_stats.items():
        print(f"\n{column}:")
        for stat, value in stats.items():
            print(f"{stat}: {value}")

    # Statistics for keywords_present_all == 0
    df_keywords_0 = df[df['keywords_present_all'] == 0]
    keywords_0_stats = calculate_stats(df_keywords_0, columns)
    print("\nStatistics for keywords_present_all == '0':")
    for column, stats in keywords_0_stats.items():
        print(f"\n{column}:")
        for stat, value in stats.items():
            print(f"{stat}: {value}")

def main():
    file_path = 'final_dataset.csv'
    df = load_and_transform_data(file_path)
    
    # Define columns for which we need statistics
    columns = ['Authentic', 'WC', 'reply_avg_tone', 'reply_avg_incivility', 'num_of_replies', 'votes']
    
    analyze_statistics(df, columns)

if __name__ == '__main__':
    main()

# Table 3 - Overlap in Authors

In [ ]:
import pandas as pd

def count_rows_across_files(files):
    """
    Counts the total number of rows across multiple CSV files.

    Parameters:
    files (list of str): List of file paths.

    Returns:
    int: Total number of rows across all files.
    """
    total_length = sum(pd.read_csv(file).shape[0] for file in files)
    return total_length

def load_and_concat_data(files, columns):
    """
    Loads CSV files and concatenates them into a single DataFrame, selecting specific columns.

    Parameters:
    files (list of str): List of file paths.
    columns (list of str): Columns to select from each CSV.

    Returns:
    DataFrame: Concatenated DataFrame with selected columns.
    """
    dfs = [pd.read_csv(file)[columns] for file in files]
    return pd.concat(dfs, ignore_index=True)

def calculate_overlap_percentages(df):
    """
    Calculates the percentage overlap of authors between subreddits.

    Parameters:
    df (DataFrame): DataFrame containing 'subreddit' and 'author' columns.

    Returns:
    dict: Dictionary with overlap percentages between subreddits.
    """
    results = {}
    subreddits = df['subreddit'].unique()
    
    for sub1 in subreddits:
        authors_sub1 = set(df[df['subreddit'] == sub1]['author'])
        total_authors_sub1 = len(authors_sub1)
        
        percentages = {}
        for sub2 in subreddits:
            if sub1 == sub2:
                continue
            authors_sub2 = set(df[df['subreddit'] == sub2]['author'])
            overlap = authors_sub1 & authors_sub2
            overlap_percentage = (len(overlap) / total_authors_sub1) * 100
            percentages[sub2] = overlap_percentage
        
        results[sub1] = percentages
    
    return results

def get_unique_authors_count(df):
    """
    Gets the number of unique authors in each subreddit.

    Parameters:
    df (DataFrame): DataFrame containing 'subreddit' and 'author' columns.

    Returns:
    dict: Dictionary with the count of unique authors per subreddit.
    """
    unique_counts = {}
    subreddits = df['subreddit'].unique()
    
    for sub in subreddits:
        authors = set(df[df['subreddit'] == sub]['author'])
        unique_counts[sub] = len(authors)
    
    return unique_counts

def save_to_csv(df, filename):
    """
    Saves a DataFrame to a CSV file.

    Parameters:
    df (DataFrame): DataFrame to save.
    filename (str): Path to the output CSV file.
    """
    df.to_csv(filename, index=False)
    print(f"Saved to {filename}")

# Define file lists
raw_files = ['raw_com_proc.csv', 'sg_com_proc.csv', 'tw_com_proc.csv', 'hk_com_proc.csv']
sub_files = ['raw_sub_proc.csv', 'sg_sub_proc.csv', 'tw_sub_proc.csv', 'hk_sub_proc.csv']

# Get the total number of rows for each set of files
total_comments_rows = count_rows_across_files(raw_files)
total_authors_rows = count_rows_across_files(sub_files)

print(f"Total number of rows across comment files: {total_comments_rows}")
print(f"Total number of rows across author files: {total_authors_rows}")

# Load, analyze, and save data for authors in subreddits
subreddit_df = load_and_concat_data(sub_files, ['subreddit', 'author'])
unique_authors_counts = get_unique_authors_count(subreddit_df)
overlap_percentages = calculate_overlap_percentages(subreddit_df)

# Create DataFrames for unique author counts and overlap percentages
unique_authors_df = pd.DataFrame(list(unique_authors_counts.items()), columns=['Subreddit', 'Unique Authors'])
overlap_data = [{'Subreddit 1': sub1, 'Subreddit 2': sub2, 'Overlap Percentage': pct}
                for sub1, percentages in overlap_percentages.items()
                for sub2, pct in percentages.items()]
overlap_df = pd.DataFrame(overlap_data)

# Save results to CSV files
save_to_csv(unique_authors_df, 'unique_authors_counts_sub.csv')
save_to_csv(overlap_df, 'overlap_percentages_sub.csv')

# Load, analyze, and save data for authors in comments
comment_df = load_and_concat_data(raw_files, ['subreddit', 'author'])
unique_authors_counts_comments = get_unique_authors_count(comment_df)
overlap_percentages_comments = calculate_overlap_percentages(comment_df)

# Create DataFrames for unique author counts and overlap percentages for comments
unique_authors_df_comments = pd.DataFrame(list(unique_authors_counts_comments.items()), columns=['Subreddit', 'Unique Authors'])
overlap_data_comments = [{'Subreddit 1': sub1, 'Subreddit 2': sub2, 'Overlap Percentage': pct}
                         for sub1, percentages in overlap_percentages_comments.items()
                         for sub2, pct in percentages.items()]
overlap_df_comments = pd.DataFrame(overlap_data_comments)

# Save results to CSV files
save_to_csv(unique_authors_df_comments, 'unique_authors_counts_comments.csv')
save_to_csv(overlap_df_comments, 'overlap_percentages_comments.csv')

# Table 4 Descriptives

In [ ]:
import pandas as pd

def process_files(file_paths, drop_na=False):
    """
    Processes a list of CSV files to calculate and print statistics.

    Parameters:
    file_paths (list of str): List of file paths to process.
    drop_na (bool): Whether to drop rows with NaN values in 'Authentic' column.
    """
    total_count = 0

    for file_path in file_paths:
        # Read the CSV file
        df = pd.read_csv(file_path)
        
        # Drop rows with NaN in 'Authentic' column if specified
        if drop_na:
            df = df.dropna(subset=['Authentic'])
        
        # Ensure the column 'keywords_present_all' is binary (0 or 1)
        df['keywords_present_all'] = df['keywords_present_all'].astype(int)
        
        # Calculate statistics
        count = (df['keywords_present_all'] == 1).sum()
        mean = df['keywords_present_all'].mean()
        std_dev = df['keywords_present_all'].std()
        num_comments = len(df)

        total_count += num_comments
        
        # Print the results for the current file
        print(f"File: {file_path}")
        print(f"  Total Number of Comments: {num_comments}")
        print(f"  Count: {count}")
        print(f"  Mean: {mean:.3f}")
        print(f"  Standard Deviation: {std_dev:.3f}")
        print()  # Print a blank line for better readability

    print(f"Total Number of Comments: {total_count}")

# File paths for initial pre-processing
initial_file_paths = [
    'raw_com_proc.csv',
    'sg_com_proc.csv',
    'tw_com_proc.csv',
    'hk_com_proc.csv'
]

# File paths for final dataset processing
final_file_paths = [
    'raw_com_proc_with_replies.csv',
    'sg_com_proc_with_replies.csv',
    'hk_com_proc_with_replies.csv',
    'tw_com_proc_with_replies.csv'
]

# Process initial files without dropping NaN values
print("Initial Pre-Processing Results:")
process_files(initial_file_paths)

# Process final files with dropping NaN values
print("Final Dataset Processing Results:")
process_files(final_file_paths, drop_na=True)

# Table 5 Descriptives

In [ ]:
import pandas as pd

def load_and_concat(file_paths):
    """
    Reads and concatenates CSV files into a single DataFrame.

    Parameters:
    file_paths (list of str): List of file paths to read and concatenate.

    Returns:
    DataFrame: Concatenated DataFrame.
    """
    dfs = [pd.read_csv(file) for file in file_paths]
    return pd.concat(dfs, ignore_index=True)

def calculate_grouped_stats(df, columns_of_interest, output_file):
    """
    Groups the DataFrame by 'subreddit', calculates mean and standard deviation,
    and exports the results to a CSV file.

    Parameters:
    df (DataFrame): DataFrame to process.
    columns_of_interest (list of str): List of columns to calculate statistics for.
    output_file (str): Path to save the results.
    """
    grouped = df.groupby('subreddit')[columns_of_interest].agg(['mean', 'std'])
    grouped = grouped.round(2).reset_index()
    grouped.to_csv(output_file, index=False)
    return grouped

def print_basic_info(df):
    """
    Prints basic information about the DataFrame.

    Parameters:
    df (DataFrame): DataFrame to get information from.
    """
    print(f'Number of comments: {len(df)}')
    print(f'Number of unique subreddits in df: {df["subreddit"].nunique()}')
    print(f'Subreddit counts in df:\n{df["subreddit"].value_counts()}')

# File paths for initial processing
initial_file_paths = [
    'hk_com_proc_with_replies_tb1.csv',
    'tw_com_proc_with_replies_tb1.csv',
    'sg_com_proc_with_replies_tb1.csv',
    'raw_com_proc_with_replies_tb1.csv'
]

# Load and process the initial data
df_before = load_and_concat(initial_file_paths)
print("Initial Data:")
print_basic_info(df_before)

# Define columns of interest
columns_of_interest = ["Authentic", "WC", "reply_avg_incivility", "num_of_replies", "votes"]

# Calculate and export statistics before dropping rows
grouped_before = calculate_grouped_stats(df_before, columns_of_interest, 'tablex1_beforedrop.csv')

# Load the final dataset
df_final = pd.read_csv('final_dataset.csv')

# Drop rows with missing values in relevant columns
df_final = df_final.dropna(subset=['Authentic', 'WC', 'is_reply', 'keywords_present_all'])

print("Final Data:")
print_basic_info(df_final)

# Calculate and export statistics after dropping rows
grouped_after = calculate_grouped_stats(df_final, columns_of_interest, 'tablex1_afterdrop.csv')

print("Exporting done!")


# Table 6 Descriptives

In [ ]:
#After Initial Pre-processing
import pandas as pd

#Obtain descriptives for Table 5
df = pd.read_csv('final_dataset.csv')

# Drop rows with missing values in the relevant columns
df = df.dropna(subset=['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
                                'Authentic', 'WC', 'is_reply', 'keywords_present_all'])

print(f"Number of Comments: {len(df)}")

#Note: 65872 of the comments do not have authenticity score

import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

def calculate_statistics(df, columns):
    # Initialize dictionary to hold the results
    stats = {}

    # Count the number of observations
    stats['count'] = df.shape[0]

    # Calculate statistics for each column
    for column in columns:
        if column in df.columns:
            stats[column] = {
                'mean': round(df[column].mean(), 2),
                'std_dev': round(df[column].std(), 2),
                'min': round(df[column].min(), 2),
                'max': round(df[column].max(), 2),
                'skewness': round(skew(df[column].dropna()), 2),
                'kurtosis': round(kurtosis(df[column].dropna()), 2)
            }
        else:
            stats[column] = 'Column not found'

    return stats

def create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent):
    # Initialize an empty DataFrame
    stats_df = pd.DataFrame()

    # Flatten the dictionary and convert it into a DataFrame
    for group_name, stats_dict in {
        'All Data': all_stats,
        'Keywords Present (1)': stats_keywords_present,
        'Keywords Absent (0)': stats_keywords_absent
    }.items():
        # Convert each group's statistics into a DataFrame
        stats_data = {k: v for k, v in stats_dict.items() if isinstance(v, dict)}
        stats_df_group = pd.DataFrame(stats_data).T
        stats_df_group['Group'] = group_name
        stats_df = pd.concat([stats_df, stats_df_group])

    # Reset the index for better readability
    stats_df.reset_index(inplace=True)
    stats_df.rename(columns={'index': 'Statistic'}, inplace=True)

    return stats_df

# Define the columns of interest
columns_of_interest = ["Authentic", "WC", "reply_avg_incivility", "num_of_replies", "votes"]

# Calculate statistics for the entire dataframe
all_stats = calculate_statistics(df, columns_of_interest)

# Calculate statistics for when df['keywords_present_all'] == 1
stats_keywords_present = calculate_statistics(df[df['keywords_present_all'] == 1], columns_of_interest)

# Calculate statistics for when df['keywords_present_all'] == 0
stats_keywords_absent = calculate_statistics(df[df['keywords_present_all'] == 0], columns_of_interest)

# Create DataFrame from the statistics
stats_df = create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent)

# Export the DataFrame to a CSV file
stats_df.to_csv('Table5_summary.csv', index=False)

print("Statistics have been exported to 'Table6_summary.csv'.")

# Table 7 Descriptives

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

def nlog(score, epsilon=1e-6):
    """
    Apply the logarithmic transformation to the given score.

    Parameters:
    score (float or array-like): The value or array of values to transform.
    epsilon (float): Small value to prevent log of zero.

    Returns:
    float or array-like: Transformed value or array of values.
    """
    return np.sign(score) * np.log(np.abs(score) + epsilon)

def calculate_statistics(df, columns):
    """
    Calculate various statistics for specified columns in the DataFrame.

    Parameters:
    df (DataFrame): DataFrame to calculate statistics on.
    columns (list of str): List of column names to calculate statistics for.

    Returns:
    dict: Dictionary containing statistics for each column.
    """
    stats = {'count': df.shape[0]}

    for column in columns:
        if column in df.columns:
            stats[column] = {
                'mean': round(df[column].mean(), 2),
                'std_dev': round(df[column].std(), 2),
                'min': round(df[column].min(), 2),
                'max': round(df[column].max(), 2),
                'skewness': round(skew(df[column].dropna()), 2),
                'kurtosis': round(kurtosis(df[column].dropna()), 2)
            }
        else:
            stats[column] = 'Column not found'

    return stats

def create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent):
    """
    Create a DataFrame from the calculated statistics.

    Parameters:
    all_stats (dict): Statistics for all data.
    stats_keywords_present (dict): Statistics where 'keywords_present_all' is 1.
    stats_keywords_absent (dict): Statistics where 'keywords_present_all' is 0.

    Returns:
    DataFrame: DataFrame containing all statistics.
    """
    stats_df = pd.DataFrame()

    stats_dict = {
        'All Data': all_stats,
        'Keywords Present (1)': stats_keywords_present,
        'Keywords Absent (0)': stats_keywords_absent
    }

    for group_name, stats_data in stats_dict.items():
        stats_df_group = pd.DataFrame(stats_data).T
        stats_df_group['Group'] = group_name
        stats_df = pd.concat([stats_df, stats_df_group])

    stats_df.reset_index(inplace=True)
    stats_df.rename(columns={'index': 'Statistic'}, inplace=True)

    return stats_df

def print_counts(df):
    """
    Print counts of comments based on different criteria.

    Parameters:
    df (DataFrame): DataFrame to get counts from.
    """
    print("Number of total comments:", len(df))
    print("Number of queer comments:", len(df[df['keywords_present_all'] == 1]))
    print("Number of non-queer comments:", len(df[df['keywords_present_all'] == 0]))
    print("Number of SGRaw comments:", len(df[df['subreddit'] == 'SingaporeRaw']))
    print("Number of SG comments:", len(df[df['subreddit'] == 'singapore']))
    print("Number of HK comments:", len(df[df['subreddit'] == 'HongKong']))
    print("Number of TW comments:", len(df[df['subreddit'] == 'taiwan']))

# Main script
def main():
    # Load dataset
    df = pd.read_csv('final_dataset.csv')

    # Drop rows with missing values in the relevant columns
    df = df.dropna(subset=['subreddit_HongKong', 'subreddit_singapore', 'subreddit_taiwan',
                            'Authentic', 'WC', 'is_reply', 'keywords_present_all'])

    # Apply the transformation function
    columns_to_transform = ["Authentic", "WC", "reply_avg_incivility", "num_of_replies", "votes"]
    for column in columns_to_transform:
        df[column] = nlog(df[column])

    # Calculate statistics
    all_stats = calculate_statistics(df, columns_to_transform)
    stats_keywords_present = calculate_statistics(df[df['keywords_present_all'] == 1], columns_to_transform)
    stats_keywords_absent = calculate_statistics(df[df['keywords_present_all'] == 0], columns_to_transform)

    # Create and export the DataFrame with statistics
    stats_df = create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent)
    stats_df.to_csv('Table6_summary.csv', index=False)
    print("Statistics have been exported to 'Table6_summary.csv'.")

    # Print counts
    print_counts(df)

if __name__ == "__main__":
    main()


# Table B1 Descriptives / Appendix B

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import skew, kurtosis

def merge_incivility(df):
    """
    Calculates the average incivility for each 'parent_id', merges this information
    back into the original dataframe, and fills missing values.
    """
    df_copy = df.copy(deep=True)
    result = df_copy.groupby('parent_id').agg({
        'incivility_prop': 'mean'
    }).reset_index()
    result['parent_id'] = result['parent_id'].str.replace(r'^t[13]_', '', regex=True)
    result = result.rename(columns={
        'parent_id': 'id', 
        'incivility_prop': 'reply_avg_incivility'
    })
    df_merged = pd.merge(df, result, on='id', how='left')
    df_merged['reply_avg_incivility'] = df_merged['reply_avg_incivility'].fillna(0)
    return df_merged

def process_files(input_files, output_files):
    """
    Process a list of input CSV files to calculate average incivility and save results to output CSV files.
    """
    for input_file, output_file in zip(input_files, output_files):
        df = pd.read_csv(input_file)
        df = merge_incivility(df)
        df.to_csv(output_file, index=False)
    print("Processing complete. Modified files saved.")

def read_and_concatenate_files(file_paths):
    """
    Read and concatenate a list of CSV files into a single DataFrame.

    Parameters:
    file_paths (list): List of file paths to read.

    Returns:
    DataFrame: Concatenated DataFrame.
    """
    dfs = [pd.read_csv(file) for file in file_paths]
    df = pd.concat(dfs, ignore_index=True)
    return df

def calculate_statistics(df, columns):
    """
    Calculate various statistics for specified columns in the DataFrame.

    Parameters:
    df (DataFrame): DataFrame to calculate statistics on.
    columns (list of str): List of column names to calculate statistics for.

    Returns:
    dict: Dictionary containing statistics for each column.
    """
    stats = {'count': df.shape[0]}
    for column in columns:
        if column in df.columns:
            stats[column] = {
                'mean': round(df[column].mean(), 2),
                'std_dev': round(df[column].std(), 2),
                'min': round(df[column].min(), 2),
                'max': round(df[column].max(), 2),
                'skewness': round(skew(df[column].dropna()), 2),
                'kurtosis': round(kurtosis(df[column].dropna()), 2)
            }
        else:
            stats[column] = 'Column not found'
    return stats

def create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent):
    """
    Create a DataFrame from the calculated statistics.

    Parameters:
    all_stats (dict): Statistics for all data.
    stats_keywords_present (dict): Statistics where 'keywords_present_all' is 1.
    stats_keywords_absent (dict): Statistics where 'keywords_present_all' is 0.

    Returns:
    DataFrame: DataFrame containing all statistics.
    """
    stats_df = pd.DataFrame()
    stats_dict = {
        'All Data': all_stats,
        'Keywords Present (1)': stats_keywords_present,
        'Keywords Absent (0)': stats_keywords_absent
    }

    for group_name, stats_data in stats_dict.items():
        stats_data = {k: v for k, v in stats_data.items() if isinstance(v, dict)}
        stats_df_group = pd.DataFrame(stats_data).T
        stats_df_group['Group'] = group_name
        stats_df = pd.concat([stats_df, stats_df_group])

    stats_df.reset_index(inplace=True)
    stats_df.rename(columns={'index': 'Statistic'}, inplace=True)
    return stats_df

def print_missing_values_summary(df, columns_to_check):
    """
    Print the number of missing values for specified columns.

    Parameters:
    df (DataFrame): DataFrame to check.
    columns_to_check (list of str): Columns to check for missing values.
    """
    missing_values_count = df[columns_to_check].isna().sum()
    print("Number of missing values for each variable:")
    print(missing_values_count)

def print_comments_summary(df):
    """
    Print summary statistics for comments based on different criteria.

    Parameters:
    df (DataFrame): DataFrame to summarize.
    """
    print(f"Number of total comments: {len(df)}")
    print(f"Number of queer comments: {len(df[df['keywords_present_all'] == 1])}")
    print(f"Number of non-queer comments: {len(df[df['keywords_present_all'] == 0])}")
    print(f"Number of SGRaw comments: {len(df[df['subreddit'] == 'SingaporeRaw'])}")
    print(f"Number of SG comments: {len(df[df['subreddit'] == 'singapore'])}")
    print(f"Number of HK comments: {len(df[df['subreddit'] == 'HongKong'])}")
    print(f"Number of TW comments: {len(df[df['subreddit'] == 'taiwan'])}")

def main():
    # Step 2: Average the replies of incivility, regardless of word count
    input_files = [
        'merged_hk_com_proc_with_replies.csv',
        'merged_tw_com_proc_with_replies.csv',
        'merged_sg_com_proc_with_replies.csv',
        'merged_raw_com_proc_with_replies.csv'
    ]
    output_files = [
        'hk_com_proc_with_replies_tb1.csv',
        'tw_com_proc_with_replies_tb1.csv',
        'sg_com_proc_with_replies_tb1.csv',
        'raw_com_proc_with_replies_tb1.csv'
    ]

    process_files(input_files, output_files)

    # Read and concatenate all processed files into a single DataFrame
    file_paths = output_files
    df = read_and_concatenate_files(file_paths)
    
    print(df.head())
    print(f"Number of rows: {df.shape[0]}")
    print(f"Number of columns: {df.shape[1]}")

    # Define columns of interest
    columns_of_interest = ["Authentic", "WC", "reply_avg_incivility", "num_of_replies", "votes"]

    # Calculate statistics
    all_stats = calculate_statistics(df, columns_of_interest)
    stats_keywords_present = calculate_statistics(df[df['keywords_present_all'] == 1], columns_of_interest)
    stats_keywords_absent = calculate_statistics(df[df['keywords_present_all'] == 0], columns_of_interest)

    # Create and export the DataFrame with statistics
    stats_df = create_statistics_df(all_stats, stats_keywords_present, stats_keywords_absent)
    stats_df.to_csv('TableB1_summary.csv', index=False)
    print("Statistics have been exported to 'TableB1_summary.csv'.")

    # Print missing values summary
    print_missing_values_summary(df, columns_of_interest)

    # Print comments summary
    print_comments_summary(df)

if __name__ == "__main__":
    main()